In [1]:
from visualization_fct import *
from sklearn.mixture import GaussianMixture
from bokeh.plotting import output_file, show, save

from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.mpl import to_bokeh
# output_file('theplott')

import matplotlib.pyplot as plt, mpld3


Html_file= open("gmm_sklearn_3components.html","w")

data = pd.read_csv("asm_data_for_ml.txt", sep='\t')
del data['MJD']
del data['error']
del data['errorA']
del data['errorB']
del data['errorC']
data['rateCA'] = data.rateC / data.rateA
data_thr = mask(data, 'orbit')  # rm too large values except for 'orbit'


# GMM with 3 components: (even without rateCA in the training data, we observe
# its particular role)
np.random.seed(1)
#X = np.c_[data_thr.orbit, data_thr.rateA, data_thr.rateB, data_thr.rateC, data_thr.rate]
X = np.c_[data_thr.orbit, data_thr.rate, data_thr.rateA, data_thr.rateB, data_thr.rateC, data_thr.rateCA]

gmm = GaussianMixture(n_components=3, n_init=5).fit(X)
preds = gmm.predict(X)
probs = gmm.predict_proba(X)

data_thr['preds'] = pd.Series(preds).astype("category")

color_key = ["red", "yellow", "blue", "grey", "black", "purple", "pink",
             "brown", "green", "orange"]  # Spectral9
color_key = color_key[:len(set(preds))+1]


In [2]:
fig = plt.figure()
sns.set()
fg = sns.pairplot(data_thr, hue="preds", palette="Set2",
                  diag_kind="kde", size=1.7)  # ,hue_order=['3', '2', '1'])


In [4]:
fg.fig.sh

In [ ]:
html = mpld3.fig_to_html(fg.fig)
html



In [ ]:
plt.figure()
plt.show()


In [ ]:
covs = gmm.covariances_
means = gmm.means_
p = scatter_matrix(data_thr, covs=covs, means=means, color_key=color_key)
show(p)